# GPT-2 Model Size Test

## 工作回顧 2021-10-01

總結之前的工作，已經可以用 pre-trianed 的 GPT-2 模型來 fine-tune，也測試過幾個文本訓練的結果，在這裡稍微總結一下心得。

### Fine-tune with New Corpus

目前測試都是基於 [CKIP 的 pretrained GPT-2](https://huggingface.co/ckiplab/gpt2-base-chinese)，再以新語料搭配 [run_clm.py](https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_clm.py) 來微調（注意：不同版本的 run_clm.py 內容略有不同）。

微調過的模型有：
- 以「古龍全集」微調
- 以「瓊瑤全集」微調
- 以「唐詩宋詞」微調
- 以「古龍全集」＋「梁羽生全集」＋「唐詩宋詞」＋「瓊瑤全集」微調

心得：
- 直接以「唐詩宋詞」訓練新的模型，生成的文字非常不流暢，微調的版本才有較通順的語意
- 微調後模型生成文字的結果，受最後訓練的文本影響很大。以前面的實驗為例，如果最後的文本是瓊瑤全集，那麼生成文字就非常瓊瑤。
- 語料中的人名、地名會被記憶住，瓊瑤全集最後一本的主角姓名就一直在生成文本裡出現。

未來可以修正的作法：
- 將長文本切割成較短的「段落」，然後隨機排列
- 將不欲學習的人名、地名以遮罩蓋掉


## 從頭訓練新模型

目前使用的 [CKIP pretrained GPT-2](https://huggingface.co/ckiplab/gpt2-base-chinese)模型，架構是使用[huggingface 預設的 GPT2Config](https://huggingface.co/transformers/model_doc/gpt2.html#gpt2config)，實際儲存空間大約 500Mb左右。這個大小的模型在 fine-tune 的過程常常會造成 GPU記憶體不足，因此我們想測試縮小模型的架構。

在[Chinese NewsTitle Generation Project by GPT2.带有超级详细注释的中文GPT2新闻标题生成项目。](https://pythonrepo.com/repo/liucongg-GPT2-NewsTitle) 這篇裡，使用了較小的模型結構：層數從 12 降到 6，模擬文字長度從 1024 降為 512。我們來測試看看實際模型大小：


In [2]:
import logging
import os
import random
import sys
import numpy as np
import tensorflow as tf
from transformers import BertTokenizerFast, TFGPT2LMHeadModel, GPT2Config
import pandas as pd

myconfig = GPT2Config(
                n_ctx=512,
                n_embd=768,
                n_head=12,
                n_layer=6,
                n_positions=512,
                vocab_size=13317
            )

tokenizer = BertTokenizerFast.from_pretrained('bert-base-chinese')
model = TFGPT2LMHeadModel(myconfig)

In [3]:
path_tokenizer = '../model/tokenizer/'
path_model = '../model/mygpt2_01'
tokenizer.save_pretrained(path_tokenizer)
model.save_pretrained(path_model)

上面這個未經訓練的模型，儲存空間大約是 7kb，讓我們用小文本稍作訓練，看看結果如何。